## Imports

In [1]:
import pandas as pd
import numpy as np
import glob
import io

import warnings
warnings.filterwarnings('ignore')

pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
pd.set_option('display.expand_frame_repr', False)
pd.set_option('max_colwidth', -1)

## Matches.csv

In [2]:
ls_df1 = [filename for filename in glob.glob("ipl_csv\*.csv")]

In [3]:
# total matches
len(ls_df1)

876

In [4]:
all_col_names = []

# match id, cols count, cols
raw_df = pd.DataFrame(index=np.arange(len(ls_df1)), columns=['m_id', 'cols_count', 'cols'])

for i in range(len(ls_df1)):
    raw_df['m_id'][i] = ls_df1[i].split('\\')[1].split('.')[0]
    
    # 22, 21, 20, 19, 18
    ls = ls_df1[i]
    try:
        temp_df = pd.read_csv(ls, nrows = 21, skiprows=1, header=None)
    except:
        try:
            temp_df = pd.read_csv(ls, nrows = 20, skiprows=1, header=None)
        except:
            try:
                temp_df = pd.read_csv(ls, nrows = 19, skiprows=1, header=None)
            except:
                try:
                    temp_df = pd.read_csv(ls, nrows = 18, skiprows=1, header=None)
                except:
                    try:
                        temp_df = pd.read_csv(ls, nrows = 17, skiprows=1, header=None)
                    except:
                        print(f"Error: {ls_df1[i]}")
                        
    # transformation to get all the columns without losing duplicates
    temp_df.columns = ['col0', 'col1', 'col2']
    mask = temp_df['col1'].duplicated(keep=False)
    temp_df.loc[mask, 'col1'] += temp_df.groupby('col1').cumcount().add(1).astype(str)
    cols_list = list(temp_df.col1)
    
    # add column count
    raw_df['cols_count'][i] = len(cols_list)
    
    # capture all column names
    all_col_names.append(cols_list)
    
    # convert list to string
    raw_df['cols'][i] = str(cols_list)

In [5]:
flat_list = [item for sublist in all_col_names for item in sublist]
all_unique_col_names = list(set(flat_list))

In [6]:
# an empty dataframe with all unique col names
df = pd.DataFrame(columns=all_unique_col_names)

In [7]:
df.head()

,date2,match_number,gender,player_of_match,season,winner_wickets,team1,city,match_referee,neutralvenue,event,team2,umpire2,method,outcome,venue,tv_umpire,reserve_umpire,winner,toss_decision,toss_winner,umpire1,date1,winner_runs,balls_per_over,date,eliminator


In [8]:
for i in range(len(ls_df1)):
    # 22, 21, 20, 19, 18
    ls = ls_df1[i]
    try:
        temp_df = pd.read_csv(ls, nrows = 21, skiprows=1, header=None)
    except:
        try:
            temp_df = pd.read_csv(ls, nrows = 20, skiprows=1, header=None)
        except:
            try:
                temp_df = pd.read_csv(ls, nrows = 19, skiprows=1, header=None)
            except:
                try:
                    temp_df = pd.read_csv(ls, nrows = 18, skiprows=1, header=None)
                except:
                    try:
                        temp_df = pd.read_csv(ls, nrows = 17, skiprows=1, header=None)
                    except:
                        print(f"Error: {ls_df1[i]}")
                        
    # transformation to get all the columns without losing duplicates
    temp_df.columns = ['col0', 'col1', 'col2']
    mask = temp_df['col1'].duplicated(keep=False)
    temp_df.loc[mask, 'col1'] += temp_df.groupby('col1').cumcount().add(1).astype(str)
    
    tempdf_keys = list(temp_df.col1)
    tempdf_values = list(temp_df.col2)                    
    
    temp_dict = dict(zip(tempdf_keys, tempdf_values))
    temp_df = pd.DataFrame([temp_dict])
    
    df = pd.concat([df, temp_df], axis = 0)

In [9]:
df.head()

,date2,match_number,gender,player_of_match,season,winner_wickets,team1,city,match_referee,neutralvenue,event,team2,umpire2,method,outcome,venue,tv_umpire,reserve_umpire,winner,toss_decision,toss_winner,umpire1,date1,winner_runs,balls_per_over,date,eliminator
0,NaN,1,male,Yuvraj Singh,2017,NaN,Sunrisers Hyderabad,Hyderabad,J Srinath,NaN,Indian Premier League,Royal Challengers Bangalore,NJ Llong,NaN,NaN,"Rajiv Gandhi International Stadium, Uppal",A Deshmukh,N Pandit,Sunrisers Hyderabad,field,Royal Challengers Bangalore,AY Dandekar,NaN,35,6,2017/04/05,NaN
0,NaN,2,male,SPD Smith,2017,7,Rising Pune Supergiant,Pune,M Nayyar,NaN,Indian Premier League,Mumbai Indians,S Ravi,NaN,NaN,Maharashtra Cricket Association Stadium,VK Sharma,Navdeep Singh,Rising Pune Supergiant,field,Rising Pune Supergiant,A Nand Kishore,NaN,NaN,6,2017/04/06,NaN
0,NaN,3,male,CA Lynn,2017,10,Gujarat Lions,Rajkot,V Narayan Kutty,NaN,Indian Premier League,Kolkata Knight Riders,CK Nandan,NaN,NaN,Saurashtra Cricket Association Stadium,YC Barde,K Srinivasan,Kolkata Knight Riders,field,Kolkata Knight Riders,Nitin Menon,NaN,NaN,6,2017/04/07,NaN
0,NaN,4,male,GJ Maxwell,2017,6,Kings XI Punjab,Indore,Chinmay Sharma,NaN,Indian Premier League,Rising Pune Supergiant,C Shamshuddin,NaN,NaN,Holkar Cricket Stadium,KN Ananthapadmanabhan,R Pandit,Kings XI Punjab,field,Kings XI Punjab,AK Chaudhary,NaN,NaN,6,2017/04/08,NaN
0,NaN,5,male,KM Jadhav,2017,NaN,Royal Challengers Bangalore,Bengaluru,J Srinath,NaN,Indian Premier League,Delhi Daredevils,VK Sharma,NaN,NaN,M.Chinnaswamy Stadium,A Nand Kishore,Navdeep Singh,Royal Challengers Bangalore,bat,Royal Challengers Bangalore,S Ravi,NaN,15,6,2017/04/08,NaN


In [10]:
# if date is missing -> add date from date1
df['date'] = df.apply(lambda x: x['date1'] if pd.isnull(x['date']) else x['date'], axis = 1)

In [11]:
# format date from string to datetime object
df['date'] = df['date'].astype('datetime64[ns]')

In [12]:
# matchId list
matchId_list = list(raw_df.m_id)

In [13]:
df['matchId'] = matchId_list

In [14]:
df.head()

,date2,match_number,gender,player_of_match,season,winner_wickets,team1,city,match_referee,neutralvenue,event,team2,umpire2,method,outcome,venue,tv_umpire,reserve_umpire,winner,toss_decision,toss_winner,umpire1,date1,winner_runs,balls_per_over,date,eliminator,matchId
0,NaN,1,male,Yuvraj Singh,2017,NaN,Sunrisers Hyderabad,Hyderabad,J Srinath,NaN,Indian Premier League,Royal Challengers Bangalore,NJ Llong,NaN,NaN,"Rajiv Gandhi International Stadium, Uppal",A Deshmukh,N Pandit,Sunrisers Hyderabad,field,Royal Challengers Bangalore,AY Dandekar,NaN,35,6,2017-04-05,NaN,1082591
0,NaN,2,male,SPD Smith,2017,7,Rising Pune Supergiant,Pune,M Nayyar,NaN,Indian Premier League,Mumbai Indians,S Ravi,NaN,NaN,Maharashtra Cricket Association Stadium,VK Sharma,Navdeep Singh,Rising Pune Supergiant,field,Rising Pune Supergiant,A Nand Kishore,NaN,NaN,6,2017-04-06,NaN,1082592
0,NaN,3,male,CA Lynn,2017,10,Gujarat Lions,Rajkot,V Narayan Kutty,NaN,Indian Premier League,Kolkata Knight Riders,CK Nandan,NaN,NaN,Saurashtra Cricket Association Stadium,YC Barde,K Srinivasan,Kolkata Knight Riders,field,Kolkata Knight Riders,Nitin Menon,NaN,NaN,6,2017-04-07,NaN,1082593
0,NaN,4,male,GJ Maxwell,2017,6,Kings XI Punjab,Indore,Chinmay Sharma,NaN,Indian Premier League,Rising Pune Supergiant,C Shamshuddin,NaN,NaN,Holkar Cricket Stadium,KN Ananthapadmanabhan,R Pandit,Kings XI Punjab,field,Kings XI Punjab,AK Chaudhary,NaN,NaN,6,2017-04-08,NaN,1082594
0,NaN,5,male,KM Jadhav,2017,NaN,Royal Challengers Bangalore,Bengaluru,J Srinath,NaN,Indian Premier League,Delhi Daredevils,VK Sharma,NaN,NaN,M.Chinnaswamy Stadium,A Nand Kishore,Navdeep Singh,Royal Challengers Bangalore,bat,Royal Challengers Bangalore,S Ravi,NaN,15,6,2017-04-08,NaN,1082595


In [15]:
# export file
df.to_csv('matches_updated.csv', index = False)

## Deliveries.csv

In [17]:
 # use any of the following column names
ls1 = ['ball', 'inning', 'over_ball', 'batting_team', 'striker', 'non_striker', 'bowler', 'batsman_runs',
           'extras', 'isWide', 'isNoBall', 'Byes', 'LegByes', 'Penalty', 'wicket_type', 'batsman_out']

ls1 = ['ball', 'inning', 'over_ball', 'batting_team', 'batsman', 'non_striker', 'bowler', 'batsman_runs',
               'extras', 'isWide', 'isNoBall', 'Byes', 'LegByes', 'Penalty', 'dismissal_kind', 'player_dismissed']

all_deliveries_df = pd.DataFrame(columns=ls1)

for i in range(len(ls_df1)):
    # fetch match id
    match_id = ls_df1[i].split('\\')[1].split('.')[0]
    
    # 22, 21, 20, 19, 18
    ls = ls_df1[i]
    
    # iterate through each match
    temp_df = pd.read_csv(ls, usecols=['version'])
    temp_df['info_ball'] = temp_df.index
    no_of_rows_to_skip = temp_df.info_ball.value_counts()['info'] + 1
    
    try:
        deliveries_temp_df = pd.read_csv(ls, skiprows=no_of_rows_to_skip, header=None)
    except:
        print(f"Error: {ls_df1[i]}")
        
   
        
    deliveries_temp_df.columns = ls1
    deliveries_temp_df['matchId'] = match_id
    all_deliveries_df = pd.concat([all_deliveries_df, deliveries_temp_df], axis = 0)
    

In [18]:
all_deliveries_df.head()

,ball,inning,over_ball,batting_team,batsman,non_striker,bowler,batsman_runs,extras,isWide,isNoBall,Byes,LegByes,Penalty,dismissal_kind,player_dismissed,matchId
0,ball,1,0.1,Sunrisers Hyderabad,DA Warner,S Dhawan,TS Mills,0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1082591
1,ball,1,0.2,Sunrisers Hyderabad,DA Warner,S Dhawan,TS Mills,0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1082591
2,ball,1,0.3,Sunrisers Hyderabad,DA Warner,S Dhawan,TS Mills,4,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1082591
3,ball,1,0.4,Sunrisers Hyderabad,DA Warner,S Dhawan,TS Mills,0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1082591
4,ball,1,0.5,Sunrisers Hyderabad,DA Warner,S Dhawan,TS Mills,0,2,2.0,NaN,NaN,NaN,NaN,NaN,NaN,1082591


In [19]:
all_deliveries_df.tail()

,ball,inning,over_ball,batting_team,batsman,non_striker,bowler,batsman_runs,extras,isWide,isNoBall,Byes,LegByes,Penalty,dismissal_kind,player_dismissed,matchId
245,ball,2,19.2,Royal Challengers Bangalore,Sachin Baby,CJ Jordan,B Kumar,2,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,981019
246,ball,2,19.3,Royal Challengers Bangalore,Sachin Baby,CJ Jordan,B Kumar,0,0,NaN,NaN,NaN,NaN,NaN,run out,CJ Jordan,981019
247,ball,2,19.4,Royal Challengers Bangalore,Iqbal Abdulla,Sachin Baby,B Kumar,0,1,NaN,NaN,NaN,1.0,NaN,NaN,NaN,981019
248,ball,2,19.5,Royal Challengers Bangalore,Sachin Baby,Iqbal Abdulla,B Kumar,1,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,981019
249,ball,2,19.6,Royal Challengers Bangalore,Iqbal Abdulla,Sachin Baby,B Kumar,4,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,981019


In [20]:
# populate over_no & ball_no
all_deliveries_df['over'] = all_deliveries_df['over_ball'].apply(lambda x: str(x).split('.')[0])
all_deliveries_df['ball'] = all_deliveries_df['over_ball'].apply(lambda x: str(x).split('.')[1])

In [21]:
all_deliveries_df.columns

Index(['ball', 'inning', 'over_ball', 'batting_team', 'batsman', 'non_striker',
       'bowler', 'batsman_runs', 'extras', 'isWide', 'isNoBall', 'Byes',
       'LegByes', 'Penalty', 'dismissal_kind', 'player_dismissed', 'matchId',
       'over'],
      dtype='object')

In [22]:
# fixing the order of all columns
all_deliveries_df =all_deliveries_df[['matchId', 'inning', 'over_ball', 'over', 'ball', 'batting_team', 'batsman', 'non_striker',
       'bowler', 'batsman_runs', 'extras', 'isWide', 'isNoBall', 'Byes',
       'LegByes', 'Penalty', 'dismissal_kind', 'player_dismissed']]

In [23]:
all_deliveries_df.head()

,matchId,inning,over_ball,over,ball,batting_team,batsman,non_striker,bowler,batsman_runs,extras,isWide,isNoBall,Byes,LegByes,Penalty,dismissal_kind,player_dismissed
0,1082591,1,0.1,0,1,Sunrisers Hyderabad,DA Warner,S Dhawan,TS Mills,0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1082591,1,0.2,0,2,Sunrisers Hyderabad,DA Warner,S Dhawan,TS Mills,0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,1082591,1,0.3,0,3,Sunrisers Hyderabad,DA Warner,S Dhawan,TS Mills,4,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,1082591,1,0.4,0,4,Sunrisers Hyderabad,DA Warner,S Dhawan,TS Mills,0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,1082591,1,0.5,0,5,Sunrisers Hyderabad,DA Warner,S Dhawan,TS Mills,0,2,2.0,NaN,NaN,NaN,NaN,NaN,NaN


In [24]:
# export file
all_deliveries_df.to_csv('deliveries_updated.csv', index = False)